In [2]:
!pip install azure.core azure.ai.formrecognizer azure.ai.documentintelligence --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.4/197.4 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.4/301.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.5/99.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.2 MB/s eta 0:00:00


In [3]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest, ContentFormat
from azure.ai.documentintelligence import DocumentIntelligenceClient

In [6]:
endpoint = ""
key = ""

# sample document
formUrl = "https://www.abim.org/Media/bfijryql/laboratory-reference-ranges.pdf"

document_analysis_client = DocumentIntelligenceClient(
    endpoint=endpoint, credential=AzureKeyCredential(key)
)

In [7]:
poller = document_analysis_client.begin_analyze_document(
    "prebuilt-layout", AnalyzeDocumentRequest(url_source=formUrl), output_content_format=ContentFormat.MARKDOWN
)
result = poller.result()

In [13]:
for table_idx, table in enumerate(result.tables):

    for cell in table.cells:
        if '1,25-Dihydroxyvitamin D' in cell.content:
            print(table_idx)
            print(
                "...Cell[{}][{}] has content '{}'".format(
                cell.row_index,
                cell.column_index,
                cell.content.encode("utf-8"),
                )
        )

0
...Cell[1][0] has content 'b'1,25-Dihydroxyvitamin D (1,25-dihydroxycholecalciferol), serum''
10
...Cell[29][0] has content 'b'Vitamin D metabolites, serum 1,25-Dihydroxyvitamin D (1,25-dihydroxycholecalciferol) 25-Hydroxyvitamin D (25-hydroxycholecalciferol)''


In [14]:
for table_idx, table in enumerate(result.tables):

    for cell in table.cells:
        if '1,5-Anhydroglucitol' in cell.content:
            print(table_idx)
            print(
                "...Cell[{}][{}] has content '{}'".format(
                cell.row_index,
                cell.column_index,
                cell.content.encode("utf-8"),
                )
        )

0
...Cell[2][0] has content 'b'1,5-Anhydroglucitol, serum or plasma''


In [27]:
import pandas as pd

final_table = pd.DataFrame()
for table_idx, table in enumerate(result.tables):
#     print(
#         "Table # {} has {} rows and {} columns".format(
#         table_idx, table.row_count, table.column_count
#         )
#     )

    new = pd.DataFrame([[0 for i in range(table.column_count)] for j in range(table.row_count)])
    for cell in table.cells:
        new.iloc[cell.row_index,cell.column_index] = cell.content
    final_table = pd.concat([final_table, new], axis=0)

<ipython-input-27-c30fab0b52af>:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Laboratory Tests' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  new.iloc[cell.row_index,cell.column_index] = cell.content
<ipython-input-27-c30fab0b52af>:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Reference Ranges' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  new.iloc[cell.row_index,cell.column_index] = cell.content
<ipython-input-27-c30fab0b52af>:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Anticardiolipin antibodies IgG IgM' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  new.iloc[cell.row_index,cell.column_index] = cell.content
<ipython-input-27-c30fab0

In [31]:
final_table.iloc[4:8,:]

,0,1
4,"17-Hydroxyprogesterone, serum",
5,"Female, follicular",<80 ng/dL
6,"Female, luteal",<285 ng/dl
7,"Female, postmenopausal",<51 ng/dl
